# Description

It combines all clustering solutions generated into a single consolidated solution using consensus clustering.

# Environment variables

In [1]:
from IPython.display import display

# set numpy n_jobs to 1, since I'll be using n_jobs later
NUMPY_N_JOBS = 1
display(NUMPY_N_JOBS)

1

In [2]:
%env MKL_NUM_THREADS=$NUMPY_N_JOBS
%env OPEN_BLAS_NUM_THREADS=$NUMPY_N_JOBS
%env NUMEXPR_NUM_THREADS=$NUMPY_N_JOBS
%env OMP_NUM_THREADS=$NUMPY_N_JOBS

env: MKL_NUM_THREADS=1
env: OPEN_BLAS_NUM_THREADS=1
env: NUMEXPR_NUM_THREADS=1
env: OMP_NUM_THREADS=1


# Modules loading

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from pathlib import Path

import numpy as np
import pandas as pd

import conf

# Settings

In [5]:
RANDOM_GENERATOR = np.random.default_rng(12345)

## Consensus clustering

In [6]:
CLUSTERING_OPTIONS = {}

CLUSTERING_OPTIONS["K_MIN"] = 2
CLUSTERING_OPTIONS["K_MAX"] = 40

display(CLUSTERING_OPTIONS)

{'K_MIN': 2, 'K_MAX': 40}

In [7]:
# output dir for this notebook
RESULTS_DIR = Path(conf.RESULTS["CLUSTERING_DIR"], "consensus_clustering").resolve()
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

display(RESULTS_DIR)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base3/results/clustering/consensus_clustering')

# Load ensemble

In [8]:
output_file = Path(RESULTS_DIR, "ensemble.npy").resolve()
display(output_file)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base3/results/clustering/consensus_clustering/ensemble.npy')

In [9]:
full_ensemble = np.load(output_file)

In [10]:
display(full_ensemble.shape)

(4428, 3749)

# Load ensemble coassociation distance matrix

In [11]:
output_file = Path(RESULTS_DIR, "ensemble_coassoc_matrix.npy").resolve()
display(output_file)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base3/results/clustering/consensus_clustering/ensemble_coassoc_matrix.npy')

In [12]:
ensemble_coassoc_matrix = np.load(output_file)

In [13]:
display(ensemble_coassoc_matrix.shape)

(3749, 3749)

In [14]:
display(ensemble_coassoc_matrix)

array([[0.        , 0.05615942, 0.05389493, ..., 0.59986413, 0.66171429,
        0.66757246],
       [0.05615942, 0.        , 0.02740036, ..., 0.60122283, 0.65988571,
        0.66530797],
       [0.05389493, 0.02740036, 0.        , ..., 0.59782609, 0.66102857,
        0.66281703],
       ...,
       [0.59986413, 0.60122283, 0.59782609, ..., 0.        , 0.6699339 ,
        0.66531165],
       [0.66171429, 0.65988571, 0.66102857, ..., 0.6699339 , 0.        ,
        0.58650558],
       [0.66757246, 0.66530797, 0.66281703, ..., 0.66531165, 0.58650558,
        0.        ]])

# Consensus clustering

In [15]:
from concurrent.futures import ProcessPoolExecutor, as_completed

from tqdm import tqdm

from clustering.ensembles.utils import (
    run_method_and_compute_agreement,
)
from clustering.ensembles.eac import (
    eac_single_coassoc_matrix,
    eac_complete_coassoc_matrix,
    eac_average_coassoc_matrix,
)
from clustering.ensembles.spectral import scc

Define spectral consensus clustering methods with delta values found in pre-analysis:

In [16]:
def scc_020(coassoc_distance_matrix, k):
    return scc(coassoc_distance_matrix, k, delta=0.20, ensemble_is_coassoc_matrix=True)


def scc_025(coassoc_distance_matrix, k):
    return scc(coassoc_distance_matrix, k, delta=0.25, ensemble_is_coassoc_matrix=True)


def scc_030(coassoc_distance_matrix, k):
    return scc(coassoc_distance_matrix, k, delta=0.30, ensemble_is_coassoc_matrix=True)


def scc_050(coassoc_distance_matrix, k):
    return scc(coassoc_distance_matrix, k, delta=0.50, ensemble_is_coassoc_matrix=True)

In [17]:
all_consensus_methods = set(
    (
        eac_single_coassoc_matrix,
        eac_complete_coassoc_matrix,
        eac_average_coassoc_matrix,
        scc_020,
        scc_025,
        scc_030,
        scc_050,
    )
)
display(all_consensus_methods)

{<function clustering.ensembles.eac.eac_average_coassoc_matrix(coassoc_matrix, k)>,
 <function clustering.ensembles.eac.eac_complete_coassoc_matrix(coassoc_matrix, k)>,
 <function clustering.ensembles.eac.eac_single_coassoc_matrix(coassoc_matrix, k)>,
 <function __main__.scc_020(coassoc_distance_matrix, k)>,
 <function __main__.scc_025(coassoc_distance_matrix, k)>,
 <function __main__.scc_030(coassoc_distance_matrix, k)>,
 <function __main__.scc_050(coassoc_distance_matrix, k)>}

In [18]:
consensus_results = []

with ProcessPoolExecutor(max_workers=conf.GENERAL["N_JOBS"]) as executor:
    tasks = {
        executor.submit(
            run_method_and_compute_agreement,
            m,
            ensemble_coassoc_matrix,
            full_ensemble,
            k,
        ): (m.__name__, k)
        for m in all_consensus_methods
        for k in range(CLUSTERING_OPTIONS["K_MIN"], CLUSTERING_OPTIONS["K_MAX"] + 1)
    }

    for future in tqdm(as_completed(tasks), total=len(tasks), disable=False, ncols=100):
        method_name, k = tasks[future]
        part, performance_values = future.result()

        method_results = {
            "method": method_name,
            "partition": part,
            "k": k,
        }
        method_results.update(performance_values)

        consensus_results.append(method_results)

  0%|                                                                       | 0/273 [00:00<?, ?it/s]

  0%|▏                                                            | 1/273 [00:25<1:53:51, 25.12s/it]

  1%|▍                                                            | 2/273 [00:26<1:21:52, 18.13s/it]

  1%|▋                                                              | 3/273 [00:28<58:40, 13.04s/it]

  1%|▉                                                            | 4/273 [00:51<1:12:44, 16.22s/it]

...

 99%|████████████████████████████████████████████████████████████▎| 270/273 [55:59<00:32, 10.95s/it]

 99%|████████████████████████████████████████████████████████████▌| 271/273 [56:11<00:22, 11.24s/it]

100%|████████████████████████████████████████████████████████████▊| 272/273 [56:30<00:13, 13.63s/it]

100%|█████████████████████████████████████████████████████████████| 273/273 [56:35<00:00, 11.00s/it]

100%|█████████████████████████████████████████████████████████████| 273/273 [56:35<00:00, 12.44s/it]

In [19]:
consensus_results = pd.DataFrame(consensus_results)

In [20]:
display(consensus_results.shape)

(273, 12)

In [21]:
consensus_results.head()

,method,partition,k,ari_mean,ari_median,ari_std,ami_mean,ami_median,ami_std,nmi_mean,nmi_median,nmi_std
0,scc_030,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2,0.225462,0.010357,0.350445,0.236081,0.054698,0.322846,0.238314,0.056364,0.322127
1,scc_030,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4,0.225381,0.010368,0.350167,0.232825,0.059509,0.304760,0.237129,0.063543,0.303123
2,scc_030,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",3,0.161708,0.063624,0.183089,0.230107,0.183277,0.117991,0.234523,0.187130,0.117597
3,scc_030,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",5,0.234826,0.013491,0.352296,0.250994,0.074993,0.303274,0.256369,0.079998,0.301111
4,scc_030,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",6,0.172859,0.061933,0.193501,0.248806,0.193558,0.134298,0.255571,0.200307,0.132916


## Testing

In [22]:
assert not consensus_results.isna().any().any()

In [23]:
# check that the number of clusters in the partitions are the expected ones
_real_k_values = consensus_results["partition"].apply(lambda x: np.unique(x).shape[0])
display(_real_k_values)
assert np.all(consensus_results["k"].values == _real_k_values.values)

0       2
1       4
2       3
3       5
4       6
       ..
268    36
269    37
270    38
271    39
272    40
Name: partition, Length: 273, dtype: int64

## Save

In [24]:
output_file = Path(RESULTS_DIR, "consensus_clustering_runs.pkl").resolve()
display(output_file)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base3/results/clustering/consensus_clustering/consensus_clustering_runs.pkl')

In [25]:
consensus_results.to_pickle(output_file)